## Calculadora de Investimentos do Aprendiz Moderno

Utilizando dados históricos para calcular a performance de uma carteira fictícia

In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import datetime as dt
pd.set_option('display.max_rows', 1000)

In [6]:
def dividendo_por_acao(lista, data):
    dividendo = lista[(lista.index.month == data.month) & (lista.index.year == data.year)]
    dividendo = dividendo.sum()
    return dividendo


def calcula_investimento(ticker, inicio='2010-01-01', termino='2020-01-01', aporte_mensal=1000.0):
    lista_de_dividendos = yf.Ticker(ticker).dividends
    hist = yf.download(ticker, start=inicio, end=termino, interval='1mo')
    hist = hist.dropna()
    hist['valor_investido'] = 0.00
    hist['aporte_mensal'] = aporte_mensal
    hist['aporte_total'] = 0.00
    hist['total_acoes'] = 0.00
    hist['resto'] = 0.00
    hist['dividendos'] = 0.00
    total_de_linhas = hist.shape[0]

    for i in range(0, total_de_linhas):
        linha = hist.iloc[i,:]
        data = hist.index[i]
        valor_de_fechamento = linha['Adj Close']
        if i > 0:
            linha_anterior = hist.iloc[i-1, :]
            acoes_anterior = linha_anterior['total_acoes']
            resto_anterior = linha_anterior['resto']
        else:
            # i == 0
            acoes_anterior = 0.00
            resto_anterior = 0.00
            
        dividendo = dividendo_por_acao(lista_de_dividendos, data)
        total_dividendos = dividendo*acoes_anterior
        aporte_total = aporte_mensal + resto_anterior + total_dividendos
        novas_acoes = int(aporte_total/valor_de_fechamento)
        total_acoes = acoes_anterior + novas_acoes
        valor_investido = total_acoes*valor_de_fechamento
        resto = aporte_total - novas_acoes*valor_de_fechamento
        # substituir valores
        hist.at[data, 'valor_investido'] = valor_investido
        hist.at[data, 'aporte_total'] = aporte_total
        hist.at[data, 'total_acoes'] = total_acoes
        hist.at[data, 'resto'] = resto
        hist.at[data, 'dividendos'] = total_dividendos

    return hist